## Steps for processing
* Convert to <a href = "https://en.wikipedia.org/wiki/Lab_color_space">l\*a\*b\* colorspace</a>. Positive values of a\* represent red/magenta color and is suitable to identify the blood samples.
* Apply <a href = "https://en.wikipedia.org/wiki/Otsu%27s_method">Otsu's thresholding</a> method to the a\* channel. As the blood samples will have higher a* values that all other portions of the image, a bimodal assumption ans hence otsu's thresholding will give us 1 for regions with high a* (most of which will be the blood samples' and zero otherwise. (a* channel is aImg, and otsu's thresholded image is otsu_img in the code)
* Find connected components in the otsu_img using connectedComponentsWithStats() function. All the blood blobs will be covered along with some other regions. In the current example, the connected components are the background (single region), 8 blobs corresponding to blood (C1's region will not be identified as it has absence of blood), and some other smaller blobs due to some spread of blood in small regions
* Using the observation mentioned above, we filter the connected regions depending upon the area. Additional criteria can be convexity, eccentricity, and the color of the connected componenents
* The centroids of the blobs are processed and each blob is assigned a vertex in the 3x3 grid. As the blob corresponding to C1 is missing, estimate its center using the rectangular property of the grid
* Use square masks for each blob and get the gray scale intensities for each of them. Solve for the standard curve estimate using linear regression on the gray scale intensities and the log-conecetration.
* Estimate the concentration of QC1,QC2 and the objective sample using the standard curve parameters

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

RuntimeError: module compiled against API version a but this version of numpy is 9

ImportError: numpy.core.multiarray failed to import

In [ ]:
inputImg = cv2.imread('image072915B.jpeg',cv2.IMREAD_COLOR)

inputImg = cv2.cvtColor(inputImg,cv2.COLOR_BGR2RGB)
grayImg = cv2.cvtColor(inputImg,cv2.COLOR_RGB2GRAY)
labImg = cv2.cvtColor(inputImg,cv2.COLOR_RGB2LAB)

In [ ]:
plt.figure(1,figsize=(10,8))
plt.imshow(inputImg)
plt.title('Input Image (RGB)')
plt.show(block=False)

In [ ]:
aImg = labImg[:,:,1] # Range of a is 0-255 (as opposed to -127 to 127); offset of 128 added
plt.figure(2,figsize=(10,8))
plt.imshow(aImg,cmap='Greys_r')
plt.title('a* channel')
plt.show(block=False)

In [ ]:
max_a_val = np.max(aImg)
min_a_val = np.min(aImg)

# print("max a* value = %f" %(max_a_val))
# print("min a* value = %f" %(min_a_val))

otsu_val,otsu_img = cv2.threshold(aImg,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)


# print("threshold a* value = %f" %(otsu_val))

In [ ]:
plt.figure(3,figsize=(10,8))
plt.imshow(otsu_img,cmap='Greys_r')
plt.title('Otsu thresholded image')
plt.show(block=False)

## Detecting blobs

In [ ]:
# Finding connected components
num,labeled_img,stats,centroids = cv2.connectedComponentsWithStats(otsu_img)

In [ ]:
print("stats (leftmost index,topmost index, width,height, area) of the connected components\n\n")
print(stats)

In [ ]:
# filtering connected components by area
areas = stats[:,4]

# desired area between 10^4 and 10^6
filtered_areas = np.logical_and(np.greater(areas,10**4), np.less(areas,10**6))

In [ ]:
blob_centers = centroids[filtered_areas]
blob_areas = areas[filtered_areas]

# check on number of blobs detected
if np.shape(blob_centers)[0]<7:
    print("Too few blobs detected")
elif np.shape(blob_centers)[0]>9:
    print("Too many blobs detected")

In [ ]:
# calculating the mean and the standard deviation for the blob centers
centroid_mean = np.mean(blob_centers,0)
centroid_x_std = np.std(blob_centers,0)

distance_threshold = centroid_x_std*.6

# coordinates relative to the blob centers
relative_centers = blob_centers-centroid_mean # x and y orientation are reversed

In [ ]:
# print(blob_centers)
# print(centroid_mean)
# print(centroid_x_std)
# print(relative_centers)
# print(distance_threshold)



## Assigning the coordinates for each grid vertex

In [ ]:
# Assigning row and column indices to each blob, and hence the center coordinates for each location in the 3*3 grid

num_blobs = np.shape(relative_centers)[0]
blob_indices = np.empty((num_blobs,2),dtype=np.int16)

grid_centers = -np.ones((3,3,2))

for i in range(0,num_blobs):
    x_val = relative_centers[i,1]
    if x_val<-distance_threshold[1]:
        blob_indices[i,0] = 0
    elif x_val>distance_threshold[1]:
        blob_indices[i,0] = 2
    else:
        blob_indices[i,0] = 1
    
    y_val = relative_centers[i,0]
    if y_val<-distance_threshold[0]:
        blob_indices[i,1] = 0
    elif y_val>distance_threshold[0]:
        blob_indices[i,1] = 2
    else:
        blob_indices[i,1] = 1
        
    grid_centers[blob_indices[i,0],blob_indices[i,1],:]=[blob_centers[i,1],blob_centers[i,0]]

In [ ]:
# Estimate centers for unrecognised blobs

# Note: Using the prior knowledge of the structure of blobs to get the top left blob center
# TODO: generalize it for any unrecognized blob


# Estimating using horizontal edge
slope1 = grid_centers[0,2,:] - grid_centers[0,1,:]
intercept1 = grid_centers[0,1,:]-slope1

# Estimating using vertical edge
slope2 = grid_centers[2,0,:] - grid_centers[1,0,:]
intercept2 = grid_centers[1,0,:]-slope2


# Averaging both the estimates
grid_centers[0,0,:] = 0.5*(intercept1+intercept2)




In [ ]:
#print(blob_indices)
#print(grid_centers[:,:,0])
#print(grid_centers[:,:,1])

## Defining mask for extracting the intensities of interest

Note: a square mask is used for each blob

In [ ]:
# Estimating the areas for a mask (using geometry to estimate area of a square bounded in a circle)
min_blob_area = np.min(blob_areas)
mask_area = min_blob_area/3.5


# defining a square mask
mask_edge = mask_area**0.5
mask_travel = int(np.floor(mask_edge/2))

In [ ]:
mask = np.zeros((np.shape(inputImg)[0],np.shape(inputImg)[1]),dtype=np.uint8)

# contains the rgb image section for each blob
mean_intensity = np.zeros((3,3))

for row_index in range(0,3):
    for col_index in range(0,3):
        x = int(np.round(grid_centers[row_index,col_index,0]))
        y = int(np.round(grid_centers[row_index,col_index,1]))
        mask[x-mask_travel:x+mask_travel,y-mask_travel:y+mask_travel] = 255
        mean_intensity[row_index,col_index] = np.mean( \
            grayImg[x-mask_travel:x+mask_travel,y-mask_travel:y+mask_travel])
    
neg_mask = 255-mask
    
masked_img = cv2.bitwise_and(inputImg,inputImg,mask=mask)
leftover_img = cv2.bitwise_and(inputImg,inputImg,mask=neg_mask)

In [ ]:
print(mean_intensity)

In [ ]:
plt.figure(4,figsize=(10,8))
plt.imshow(leftover_img)
plt.title('Masked regions')
plt.show(block=False)

##  Estimating the standard curve

In [ ]:
# Defining conc Matrix using known value
# conc_mat = np.array([[0,np.nan,1000],[62.5,np.nan,500],[125,np.nan,250]])
conc_mat = np.array([[0,np.nan,1000],[62.5,np.nan,500],[125,np.nan,250]])

logconc_mat = np.log(conc_mat)

In [ ]:
# Solving the linear equation between intensity and log-concentration (Srandard curve)

# counting the number of conc values available for linear regression
valid_list = list()
for row_index in range(0,3):
    for col_index in range(0,3):
        if not(np.isinf(logconc_mat[row_index,col_index]) or np.isnan(logconc_mat[row_index,col_index])):
            valid_list.append((row_index,col_index))

In [ ]:
num_valid = len(valid_list)

# Performing linear regression
xData = np.empty(num_valid)
yData = np.empty(num_valid)

for i in range(0,num_valid):
    row_index, col_index = valid_list[i]
    yData[i] = mean_intensity[row_index,col_index]
    xData[i] = logconc_mat[row_index,col_index]


In [ ]:
# Solving the linear equation
A = np.vstack([xData, np.ones(len(xData))]).T
m, c = np.linalg.lstsq(A, yData)[0]

linear_inp = np.linspace(logconc_mat[1,0],logconc_mat[0,2],100)
linear_out = m*linear_inp+c

In [ ]:
print("Slope = %f" %m)
print("intercept = %f" %c)

print(logconc_mat)

In [ ]:
plt.figure(6,figsize=(10,8))
plt.scatter(xData,yData)
plt.plot(linear_inp,linear_out)
plt.xlabel('log concentration')
plt.ylabel('Intensities')
plt.title('Scatter plot and estimated standard curve')
# Plotting the regression result
plt.show(block=False)

In [ ]:
# Estimating on control points

# solving for QC1
exp_val_qc1 = 156
row_index = 2
col_index = 1

intensity = mean_intensity[row_index,col_index]
logconc = (intensity-c)/m
calc_val_qc1 = np.exp(logconc)


# solving for QC2
exp_val_qc2 = 750
row_index = 0
col_index = 1

intensity = mean_intensity[row_index,col_index]
logconc = (intensity-c)/m
calc_val_qc2 = np.exp(logconc)


# Calculating deviation in values obtained
dev_1 = abs((exp_val_qc1-calc_val_qc1)/(exp_val_qc1))
dev_2 = abs((exp_val_qc2-calc_val_qc2)/(exp_val_qc2))

In [ ]:
print('Calculated intensity for QC1: %f' %(calc_val_qc1))
print('Calculated intensity for QC1: %f' %calc_val_qc2)

print('percent deviation for QC1: %f' %(dev_1))
print('percent deviation for QC2: %f' %(dev_2))

In [ ]:
plt.show()